In [1]:
from pyod.models.ecod import ECOD  # ECOD detector
from pyod.models.knn import KNN   # kNN detector
from joblib import dump, load
from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize
from loader import Loader
import os

In [2]:
clf_name = "KNN"
# clf = ECOD()
clf = KNN()


In [3]:
loader = Loader()
if not os.path.exists("data/data.pkl"):
    loader.extract()
    df = loader.create_df_from_xlsx()
    df = loader.process_df(df)
    loader.save_df(df)
else:
    df = loader.load_df()


Processing: 100%|██████████| 12872/12872 [00:15<00:00, 823.15it/s]


In [6]:
print(df.shape)
print(df.columns)
print(df.head())

(12872, 19)
Index(['کد محور', 'مدت زمان کارکرد (دقیقه)', 'تعداد کل وسیله نقلیه',
       'تعداد وسیله نقلیه کلاس ۱', 'تعداد وسیله نقلیه کلاس ۲',
       'تعداد وسیله نقلیه کلاس ۳', 'تعداد وسیله نقلیه کلاس ۴',
       'تعداد وسیله نقلیه کلاس ۵', 'سرعت متوسط', 'تعداد تخلف سرعت غیر مجاز',
       'تعداد تخلف فاصله غیر مجاز', 'تعداد تخلف سبقت غیر مجاز',
       'تعداد برآورد شده', 'تعداد وسیله نقلیه سبک', 'تعداد وسیله نقلیه سنگین',
       'holiday', 'date', 'day_of_week', 'time_start'],
      dtype='object')
   کد محور  مدت زمان کارکرد (دقیقه)  تعداد کل وسیله نقلیه  \
0   113201                     1440                  1503   
1   113201                     1380                  1589   
2   113201                     1440                  1767   
3   113201                     1430                  2221   
4   113201                     1440                  2379   

   تعداد وسیله نقلیه کلاس ۱  تعداد وسیله نقلیه کلاس ۲  \
0                    1302.0                      66.0   
1             

In [5]:
df[df["holiday"] == True]


,کد محور,مدت زمان کارکرد (دقیقه),تعداد کل وسیله نقلیه,تعداد وسیله نقلیه کلاس ۱,تعداد وسیله نقلیه کلاس ۲,تعداد وسیله نقلیه کلاس ۳,تعداد وسیله نقلیه کلاس ۴,تعداد وسیله نقلیه کلاس ۵,سرعت متوسط,تعداد تخلف سرعت غیر مجاز,تعداد تخلف فاصله غیر مجاز,تعداد تخلف سبقت غیر مجاز,تعداد برآورد شده,تعداد وسیله نقلیه سبک,تعداد وسیله نقلیه سنگین,holiday,date,day_of_week,time_start
0,113201,1440,1503,1302.0,66.0,15.0,32.0,88.0,101.00,17,47,0,1503.0,NaN,NaN,True,1395-01-01,1,00:00:00
1,113201,1380,1589,1349.0,74.0,26.0,50.0,90.0,100.00,14,51,0,1614.0,NaN,NaN,True,1395-01-02,2,00:00:00
2,113201,1440,1767,1408.0,138.0,34.0,59.0,128.0,99.00,18,57,0,1767.0,NaN,NaN,True,1395-01-03,3,00:00:00
3,113201,1430,2221,1709.0,196.0,51.0,89.0,176.0,99.00,17,79,0,2233.0,NaN,NaN,True,1395-01-04,4,00:00:00
11,113201,1440,3298,2501.0,219.0,107.0,141.0,330.0,99.00,18,155,0,3298.0,NaN,NaN,True,1395-01-12,5,00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,114552,1365,5686,5414.0,172.0,2.0,21.0,77.0,95.27,1310,0,0,NaN,NaN,NaN,True,1395-01-02,2,00:00:00
2,114552,1440,6163,5796.0,223.0,3.0,21.0,120.0,87.95,832,1,0,NaN,NaN,NaN,True,1395-01-03,3,00:00:00
3,114552,1425,5982,5186.0,659.0,5.0,29.0,103.0,94.41,1239,0,0,NaN,NaN,NaN,True,1395-01-04,4,00:00:00
11,114552,1440,8607,7700.0,697.0,14.0,50.0,146.0,88.34,1062,0,0,NaN,NaN,NaN,True,1395-01-12,5,00:00:00


In [7]:
print(loader.get_roads())

{113201: 'آزادراه قم - گرمسار (چرمشهر)', 113202: 'شهريار-کرج (هفت جوي)', 113203: 'آزادراه تهران - قم (مجتمع ياس)', 113204: 'آزادراه تهران - ساوه (عوارضي تهران)', 113205: 'جاجرود - بومهن', 113206: 'آزادراه تهران - پرديس (عوارضي پرديس)', 113207: 'آزادراه تهران - پرديس (بومهن)', 113208: 'آزادراه ساوه - تهران (رباط کريم-تهران)', 113209: 'تهران - پاکدشت', 113210: 'آزادراه تهران - قم (عوارضي تهران)', 113211: 'چرمشهر - حسن\u200cآباد', 113212: 'امام زاده داوود - تهران', 113213: 'تهران - لواسانات (طلائيه)', 113214: 'تهران - لواسانات (سوهانک)', 113228: 'آزادراه تهران - ساوه (پرند)', 113243: 'جاده قديم تهران، قم (حسن\u200cآباد - نعلبندان)', 113249: 'آزادراه تهران- شمال (قطعه اول - محدوده عوارضي تهران)', 113251: 'آزادراه گرمسار - قم (چرمشهر)', 113252: 'کرج-شهريار (هفت جوي)', 113253: 'آزادراه قم - تهران (مجتمع ياس)', 113254: 'آزادراه ساوه - تهران (عوارضي تهران)', 113255: 'بومهن - جاجرود', 113256: 'آزادراه پرديس - تهران (عوارضي پرديس)', 113257: 'آزادراه پرديس - تهران (بومهن)', 113258: 'آزادراه تهران

In [ ]:
# TODO
X_train = []

In [ ]:
clf.fit(X_train)

# get the prediction label and outlier scores of the training data
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

# get the prediction on the test data
y_test_pred = clf.predict(X_test)  # outlier labels (0 or 1)
y_test_scores = clf.decision_function(X_test)  # outlier scores

# it is possible to get the prediction confidence as well
y_test_pred, y_test_pred_confidence = clf.predict(
    X_test, return_confidence=True
)  # outlier labels (0 or 1) and confidence in the range of [0,1]


In [ ]:
# evaluate and print the results
print("\nOn Training Data:")
evaluate_print(clf_name, y_train, y_train_scores)
print("\nOn Test Data:")
evaluate_print(clf_name, y_test, y_test_scores)


In [ ]:
visualize(
    clf_name,
    X_train,
    y_train,
    X_test,
    y_test,
    y_train_pred,
    y_test_pred,
    show_figure=True,
    save_figure=False,
)


In [ ]:
# save the model
dump(clf, "clf.joblib")
# load the model
clf = load("clf.joblib")
